# OpenMendel: Iterative Hard Thresholding Tutorial

### Last update: 11/3/2018

### Julia version

For reproducibility, the computer spec and Julia version is listed below. Current code supports Julia version 0.6.4. It will not work in v0.7 or v1.0, but upgrade to v1.0 is very high on our TODO list. 

The IHT.jl module must be installed on your computer before running this tutorial.  Instructions on cloning and the latest IHT code can be found here: https://github.com/biona001/IHT.jl

In [1]:
versioninfo()

Julia Version 0.6.4
Commit 9d11f62bcb (2018-07-09 19:09 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-3740QM CPU @ 2.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge MAX_THREADS=16)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, ivybridge)


### When to use Iterative Hard Thresholding

*Continuous* model selection is advantageous in situations where the multivariate nature of the regressors plays a significant role *together*. As an alternative to traditional SNP-by-SNP association testing, iterative hard-thresholing (IHT) performs continuous model selection on a GWAS dataset $\mathbf{X} \in \{0, 1, 2\}^{n \times p}$ and continuous phenotype vector $\mathbf{y}$ by minimizing the residual sum of squares $f(\beta) = \frac{1}{2}||\mathbf{y} - \mathbf{X}\beta||^2$ subject to the constraint that $\beta$ is $k-$sparse. This method has the edge over LASSO (which also provides continuous model selection) because IHT does not shrink estimated effect sizes. Parallel computing is offered through `q-`fold cross validation, and in the near future, dense (genotype matrix)-(dense vector) multiplication. 

### Appropriate Datasets and Example Inputs 

All genotype data **must** be stored in the [PLINK binary genotype format](https://www.cog-genomics.org/plink2/formats#bed), where the triplets `.bim`, `.bed` and `.fam` must all be present. Additional non-genetic covariates should be stored in a separate file (e.g. comma separated file). In the first 3 examples of this tutorial, we use "gwas 1 data" (github repo: [here](https://github.com/OpenMendel/MendelGWAS.jl/tree/master/docs)) to illustrate the basic usage and functionalities of MendelIHT. This dataset has 2200 people and a modest 10000 simulated SNPs, with 2 SNPs (`rs1935681` and `rs2256412`) contributing to the response. One can obtain this dataset from the first example input of [MendelGWAS.jl](https://openmendel.github.io/MendelGWAS.jl/), or via option 24a of the free application [Mendel version 16](http://www.genetics.ucla.edu/software/mendel). To examine the robustness and accuracy of IHT, examples 4 and 5 simulates data on the spot, and immediately calls IHT on the simulated data. 


### Missing Data

`MendelIHT` assumes there are no missing genotypes, since it uses linear algebra functions defined in [SnpArrays.jl](https://openmendel.github.io/SnpArrays.jl/latest/man/snparray/#linear-algebra-with-snparray). Therefore, you must first impute missing genotypes before you use MendelIHT. SnpArrays.jl offer a naive imputation strategy, but otherwise, our own software [option 23 of Mendel](http://www.genetics.ucla.edu/software/mendel) is a reasonable choice. Open Mendel will soon provide a separate package `MendelImpute` containing new imputation strategies such as alternating least squares.  

### Cross Validation and Regularization paths

We usually have very little information on how many SNPs are affecting the phenotype. In a typical GWAS study, anywhere between 1 to thousands of SNPs could play a role. Thus ideally, we can test many different models to find the best one. MendelIHT provides 2 ways for one to perform this automatically: user specified regulartization paths, and $q-$fold [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics). Users should know that, in the first method, increasing the number of predictors will almost always decrease the error, but as a result introduce overfitting. Therefore, in most practical situations, it is highly recommended to combine this method with cross validation. In $q-$fold cross validation, samples are divided into $q$ disjoint subsets, and IHT fits a model on $q-1$ of those sets data, then computes the [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) tested on the $qth$ samples. Each $q$ subsets are served as the test set exactly once. With noisy responses, IHT cross validation typically underestimates the true model size, as shown in the examples below. 

### Analysis keywords available to users 

| Keyword | Default Value | Allowed value | Description |
| --- | --- | --- | --- |
|`predictors` | 0 | Positive integer | Max number of non-zero entries of $\beta$ |
|`non_genetic_covariates` | "" | File name on disk | Delimited file containing the non-genetic covariates for each sample |
|`run_cross_validation` | false | boolean | Whether the user wants to run cross-validation |
|`model_sizes` | "" | Integers stored in string separated by ',' | Different model sizes users wish to run IHT |
|`cv_fold` | 0 | Positive integer | Number of disjoint subsets the samples should be divided into |
|`max_groups` (\*) | 1 | Integer | Total number of groups |
|`group_membership` (\*) | "" | File name on disk | File indicating group membership |
|`prior_weights` (\*) | "" | maf | How to scale predictors based on different weights |

+ (\*) Indicates experimental features. We currently have no theoretical guarantees on their performance, therefore illustrations of these functionalities are omitted from this tutorial. Users should tread carefully with these features. 
+ A list of OpenMendel keywords common to most analysis package can be found [here](https://openmendel.github.io/MendelBase.jl/#keywords-table)

# Example 1: Run IHT with Only Genotype Data

### Step 1: Preparing Input files

In Open Mendel, all analysis parameters are specified via the [Control file](https://openmendel.github.io/MendelBase.jl/#control-file). Genotype data must be inputted via the PLINK binary format. The most basic control file to run IHT looks like the following:

In [2]:
;cat "gwas 1 Control basic.txt"

#
# Input and Output files.
#
plink_input_basename = gwas 1 data

#
# Analysis parameters for IHT option.
#
predictors = 2

### Step 2: Run MendelIHT

To run `MendelIHT`, execute the following in the Julia REPL or in this notebook:

In [3]:
using IHT
MendelIHT("gwas 1 Control basic.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.4.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/docs/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control basic.txt
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Analyzing the data for model size k = 2


IHT results:

Compute time (sec):     1.477848303
Final loss:             1183.6890803583537
Iterations:             4
Max number of groups:   1
Max predictors/group:   2
IHT estimated 2 nonzero coefficients.
2×3 DataFrames.DataFrame
│ Row │ Group │ Predictor │ Estimated_β │
├─────┼───────┼───────────┼─────────────┤
│ 1   │ 1     │ 3981      │ 0.149073    │
│ 2   │ 1     │ 7023      │ 0.272991    │

Intercept of model = 0.0


### Step 3: Interpreting the results

Here the estimated model is the 3981th and 7023th predictor, corresponding to rs1935681 and rs2256412 in the `gwas 1 data.bim` file, which are the correct SNPs. The intercept of the model is given at the bottom of the table. Here the compute time is the time associated with computing the optimal $\beta$ only, so it does not include other necessary processes, such as importing the data. 

**Note:** the `affected_designator = 2` simply indicates that the pedigree is a Plink .fam file, which must always be the case for `MendelIHT` because this analysis option only accepts PLINK binary format as inputs. 

# Example 2: Including Non-Genetic Covariates

Non-genetic covariates must be stored in a comma demited file, with the same number of rows as the number of samples. The intercept term (i.e. grand mean) must also be included in the file. If the user does not specify a non-genetic covariate file, `MendelIHT` will by default include an intercept in the estimated model. 

### Step 1: Prepare Non-Genetic Covariate File

In this example, we generated one non-genetic covariate from a $N(0, 1)$ distribution. After including the grand mean, we saved the file in `gwas 1 noncov.txt` where the entries are separated by a tab. The first few lines of this file looks like the following:

In [4]:
;head -10 "gwas 1 noncov.txt"

1	-0.088704513339476
1	-0.9575873240069772
1	-0.9713258274139007
1	-0.9847900613424241
1	-0.5954781589540936
1	0.2124813875751884
1	2.28150775802523
1	1.7643235366779797
1	-0.3933262467789896
1	-0.1348394065324508


### Step 2: Prepare Corresponding Control File 

We need to tell MendelIHT that the covariates are separated by tabs. This can be specified via the [MendelBase](https://openmendel.github.io/MendelBase.jl/) keyword `field_separator` in the control file as follows:

In [5]:
;cat "gwas 1 Control nongen.txt"

#
# Input and Output files.
#
plink_input_basename = gwas 1 data
non_genetic_covariates = gwas 1 noncov.txt
field_separator = '	'
#
# Analysis parameters for IHT option.
#
predictors = 2

### Step 3: Run IHT 

In [6]:
using IHT
MendelIHT("gwas 1 Control nongen.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.4.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/docs/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control nongen.txt
  field_separator = 	
  non_genetic_covariates = gwas 1 noncov.txt
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Analyzing the data for model size k = 2


IHT results:

Compute time (sec):     0.979895101
Final loss:             1183.6890803583544
Iterations:             4
Max number of groups:   1
Max predictors/group:   2
IHT estimated 2 nonzero coefficients.
2×3 DataFrames.DataFrame
│ Row │ Group │ Predictor │ Estimated_β │
├─────┼───────┼───────────┼─────────────┤
│ 1   │ 1     │ 3981      │ 0.149073    │
│ 2   │ 1     │ 7023      │ 0.272991    │

Intercept of model = 0.0


**Remark:** Observe that the resulting error and model is exactly the same, because the covariates we added is white noise, and thus, was not selected by IHT to be a significant predictor.

# Example 3: Cross Validation

In this example, users can run IHT on any number of model in at most 8 parallel threads. Empirically, running on $n$ threads achieves roughly $n/2$ fold speedup. Note that running $q$ fold cross validation on $r$ different models entails running IHT $q \times r$ times. 

### Step 0: IMPORTANT in order to take advantage of multithreads, discontinue this notebook and julia. Have a terminal open.

### Step 1: IMPORTANT Execute following line in the terminal BEFORE starting notebook (or Julia REPL) 
export JULIA_NUM_THREADS=8

### Step 2: Start notebook in the same terminal window and verify that notebook is indeed running with 8 threads:  
Note that if you computer's capacity is less than 8, it will default to the largest number it can run. 

In [7]:
Threads.nthreads()

8

### Step 3: Specify the model sizes

The paths should be inside quotes and separated by comma, specified via the keyword `model_sizes`. Each entry must be an integer. In this example, we tried to run IHT for model sizes $k = 1, 2, ..., 10$ and 5 different folds. This is equivalent to running IHT 50 different times, and hence, ideal for parallel computing. 

In [8]:
;cat "gwas 1 Control cv.txt"

#
# Input and Output files.
#
plink_input_basename = gwas 1 data
#
# Cross Validation parameters
#
run_cross_validation = true
cv_folds = 5
model_sizes = "1,2,3,4,5,6,7,8,9,10"


### Step 4: Run Cross Validation to find best model size

In [9]:
using IHT
MendelIHT("gwas 1 Control cv.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.4.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/docs/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control cv.txt
  cv_folds = 5
  model_sizes = 1,2,3,4,5,6,7,8,9,10
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  run_cross_validation = true
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected




Crossvalidation Results:
k	MSE
1	0.5546784765054349
2	0.6004528460555323
3	0.6660062486666182
4	0.7124842256809767
5	0.7284745101298649
6	0.6771718567273666
7	0.6298517451998119
8	0.6174543457147491
9	0.6406876914640967
10	0.6628730245475413

The lowest MSE is achieved at k = 1


INFO: Running 5-fold cross validation on the following model sizes:
1,2,3,4,5,6,7,8,9,10.
Ignoring keyword predictors.


### Step 5: Re-run ordinary IHT on the best model size

According to our cross validation result, the best model size that minimizes out-of-sample errors (i.e. MSE on the q-th subset of samples) is attained at $k = 1$. Using this information, one can re-run the IHT code to obtain the estimated model. Note importantly, that cross-validation in this case **did not** return the true model size $k = 2$. This reflects the general tendency for IHT to *under* estimate the true model size.

In [10]:
;cat "gwas 1 Control basic2.txt"

#
# Input and Output files.
#
plink_input_basename = gwas 1 data

#
# Analysis parameters for IHT option.
#
predictors = 1

In [11]:
MendelIHT("gwas 1 Control basic2.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.4.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/docs/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control basic2.txt
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 1
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Analyzing the data for model size k = 1


IHT results:

Compute time (sec):     0.897766462
Final loss:             1208.1108390895013
Iterations:             3
Max number of groups:   1
Max predictors/group:   1
IHT estimated 1 nonzero coefficients.
1×3 DataFrames.DataFrame
│ Row │ Group │ Predictor │ Estimated_β │
├─────┼───────┼───────────┼─────────────┤
│ 1   │ 1     │ 7023      │ 0.276275    │

Intercept of model = 0.0


### Step 6 (optional): Running different path sizes but don't run cross validation

Instead of cutting your sample size into $q$ disjoint sets, IHT optionally allows users to run various models on all the samples. This happens if the user does not set `run_cross_validation` to true. An example is as follows:

In [12]:
;cat "gwas 1 Control userpath.txt"

#
# Input and Output files.
#
plink_input_basename = gwas 1 data
#
# Analysis parameters for IHT option.
#
predictors = 2
model_sizes = "1,2,3,4,5,6,7,8,9,10"


In [13]:
using IHT
models, model_errors = MendelIHT("gwas 1 Control userpath.txt")

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.4.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/docs/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control userpath.txt
  model_sizes = 1,2,3,4,5,6,7,8,9,10
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Running the following model sizes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.140544 0.140545])

**Remark:** We can check that IHT is stable given different sparsity constraints, in the sense that parameters selected with a sparser constraint will still be selected if given a weaker constraint. Below we list the 10 models in separate columns.  

In [14]:
idx = find(models[:, end])
full(models[idx, :])

9×10 Array{Float64,2}:
 0.0       0.0       0.0       …  -0.0756696  -0.0766181  -0.0782086
 0.0       0.0       0.0          -0.0780773  -0.077302   -0.0787067
 0.0       0.149073  0.149073      0.145485    0.144214    0.143389 
 0.0       0.0       0.0           0.08293     0.080625    0.0810577
 0.0       0.0       0.0          -0.087082   -0.0869424  -0.0890821
 0.0       0.0       0.0       …   0.0866336   0.0857555   0.0875377
 0.0       0.0       0.0           0.0        -0.0737263  -0.0751238
 0.276275  0.272991  0.272991      0.265817    0.264556    0.2656   
 0.0       0.0       0.0           0.0         0.0        -0.0832995

# Example 4: Using simulated results to examine IHT signal reconstruction

In this example, we want to explicitly examine the ability for IHT to recover signals. Note the follow 2 examples directly call functions of `IHT.jl` instead of using control files. This allows greater flexibility on end users because they can directly manipulate variables. 

### Step 1: Simulate data

We first simulate a uniform random SNP matrix $X$ and a known model $\beta$ as follows:
$$\beta \sim N(0, 1),$$
$$y = X\beta + \epsilon$$
$$\epsilon \in N(0, s),$$
$$s \in \{0.1, 1, 10, 25\}.$$

In [15]:
#load packages
using IHT
using SnpArrays
using DataFrames
using Distributions

#set random seed
srand(1111) 

#specify dimension and noise of data
n = 5000                        # number of cases
p = 30000                       # number of predictors
k = 10                          # number of true predictors per group
S = [0.1, 1.0, 10.0, 25.0]      # noise vector, from very little noise to a lot of noise

#construct snpmatrix, covariate files, and true model b
x           = SnpArray(rand(0:2, n, p))    # a random snpmatrix
z           = ones(n, 1)                   # non-genetic covariates, just the intercept
true_b      = zeros(p)                     # model vector
true_b[1:k] = randn(k)                     # Initialize k non-zero entries in the true model
shuffle!(true_b)                           # Shuffle the entries
correct_position = find(true_b)            # keep track of what the true entries are
noise = [rand(Normal(0, s), n) for s in S] # noise vectors from N(0, s) where s ∈ S = {0.01, 0.1, 1, 10}s

#compute mean and std used to standardize data to mean 0 variance 1
mean_vec, minor_allele, = summarize(x)
for i in 1:p
    minor_allele[i] ? mean_vec[i] = 2.0 - 2.0mean_vec[i] : mean_vec[i] = 2.0mean_vec[i]
end
std_vec = std_reciprocal(x, mean_vec)

#simulate phenotypes under different noises by: y = Xb + noise 
y = [zeros(n) for i in 1:length(S)]
for i in 1:length(S)
    SnpArrays.A_mul_B!(y[i], x, true_b, mean_vec, std_vec)
    y[i] .+= noise[i]
end

### Observe the noisiness of our observations. 
Each column is the same response vector with varying levels of noise added. The farther right, the noisier the data.

In [16]:
[y[1] y[2] y[3] y[4]]

5000×4 Array{Float64,2}:
 -2.99018    -2.22532    -1.82439    6.86836
  0.0639139  -0.369665    5.09387    5.69815
  0.842478    0.752519   18.7169   -28.0052 
 -2.82943    -3.8914      9.95096    4.61981
 -1.67051    -0.116633   -4.05996  -33.7148 
 -0.514721   -1.95655    -6.08555  -36.0566 
 -0.801959   -1.37353    -7.47094  -16.0338 
  2.96974     5.17726    -8.5696   -12.3472 
  3.10826     2.3005      7.51206  -33.1643 
 -0.464412    0.4904     -5.58351   16.1176 
 -0.616762   -0.790245   10.3874   -21.1504 
  0.626816    0.374474   -8.41267   31.0773 
 -2.15693    -1.69911   -10.2978    23.1543 
  ⋮                                         
 -0.848227   -1.33377     6.24567  -25.5871 
  1.77269     2.75435     1.8081   -14.2573 
  3.28046     2.37263   -10.7767     2.74097
  2.53484     3.85943   -14.6097   -27.4922 
 -0.28382     1.24455     6.40927  -20.1135 
  2.57121     3.26019    10.9377   -28.2579 
  0.720075    0.407501  -14.2108   -24.9081 
 -1.00212    -0.294159    2.86

### Step 2: Examine reconstruction of IHT given true model size

In [17]:
#compute IHT result for less noisy data
estimated_models = [zeros(k) for _ in 1:length(y)]
for i in 1:length(y)
    v = IHTVariables(x, z, y[i], 1, k)
    result = L0_reg(v, x, z, y[i], 1, k)
    estimated_models[i] .= result.beta[correct_position]
end

#compare and contrast
true_model = true_b[correct_position]
compare_model = DataFrame(
    correct_position = correct_position, 
    true_β           = true_model, 
    noise_level_1    = estimated_models[1],      #N(0, 0.1)
    noise_level_2    = estimated_models[2],      #N(0, 1.0)
    noise_level_3    = estimated_models[3],      #N(0, 10.0)
    noise_level_4    = estimated_models[4])      #N(0, 25.0)

,correct_position,true_β,noise_level_1,noise_level_2,noise_level_3,noise_level_4
1,2012,0.20775,0.208856,0.205502,0.0,0.0
2,5685,0.372181,0.373598,0.357529,0.629893,0.0
3,7835,0.416006,0.414099,0.401277,0.0,0.0
4,9271,-0.160673,-0.161993,-0.168774,0.0,0.0
5,13188,-0.545489,-0.546157,-0.526095,0.0,0.0
6,19042,-0.695873,-0.696818,-0.714161,-0.672233,0.0
7,21966,-0.202504,-0.202652,-0.217099,0.0,0.0
8,23982,-0.489064,-0.489366,-0.495835,-0.56097,0.0
9,24758,0.551161,0.553689,0.529186,0.0,0.0
10,29481,-1.56535,-1.56519,-1.55339,-1.73005,-1.79669


### Step 3: Interpret reconstruction result

IHT finds the correct 10 predictors if we add little noise. With greater noise, we lose predictors with smaller effect size, while losing accuracy. However, found predictors exhibit no shrinkage of effect size. 

# Example 5: Using simulated results to examine IHT Cross Validation

### Step 0: Follow steps 0~2 in example 3 to gain access to multi-threading

In [18]:
Threads.nthreads() # check that multiple threads are running

8

### Step 1: Simulate Data (as done in Example 4 step 1)

### Step 2: Run IHT cross validation code

In [19]:
srand(1111) #reset seed

#run cross validation on not-so-noisy data
path      = collect(2:2:20) 
num_folds = 3
folds     = rand(1:num_folds, n)
cv_iht(x, z, y[1], 1, path, folds, num_folds)



Crossvalidation Results:
k	MSE
2	3.1677822019919044
4	2.8842864963598736
6	2.831063934427872
8	1.8713059507183205
10	2.257791657013664
12	2.2743936672699987
14	2.2656904985204647
16	2.263997497534819
18	2.2599121194306795
20	2.2535873590855027

The lowest MSE is achieved at k = 8


In [20]:
srand(1111) #reset seed

#run cross validation on quite noisy data
path      = collect(2:2:20) 
num_folds = 3
folds     = rand(1:num_folds, n)
cv_iht(x, z, y[3], 1, path, folds, num_folds)



Crossvalidation Results:
k	MSE
2	53.6730684917142
4	53.65755181959526
6	53.93280154016717
8	54.69513090164977
10	54.86837034931569
12	53.55242920512205
14	53.83447900670268
16	55.6527829783363
18	56.986528750175154
20	57.14268507459773

The lowest MSE is achieved at k = 12


### Step 3: Interpreting results (recall $k_{\text{true}} = 10$)

When noise $\epsilon \in N(0, 0.1)$, cross-validation slightly underestimates the true sparsity parameter: $k_{\text{estimate}} = 8$. For very noisy data $\epsilon \in N(0, 10)$, cross validation returns $k_{\text{estimate}} = 12$, which slightly over-estimates the true sparsity parameter.

# Conclusion

This notebook demonstrated some of the basic features of IHT. For more examples (particularly on how to use IHT on numeric data), readers can visit our older notebooks [here](https://github.com/klkeys/IHT.jl/blob/master/docs/IHT_tutorial_full.ipynb) and [here](https://github.com/klkeys/IHT.jl/blob/master/docs/IHT_tutorial_brief.ipynb). The first 3 examples illustrate the basic usage of IHT for general audiences. The last 2 examples illustrate how to make function calls directly and tests its robustness on reproducible simulated data. Notably, we selectively omitted several experimental features of IHT. In the near future, we will release a more detailed notebook that includes tutorials on all the experiemental features of IHT on GWAS data as well as motivations for them.